In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_1.csv",r"B:\project\ai\ai_chunk_1.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)

(30580, 2)


In [7]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essay=df['text'].tolist()
tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)


In [8]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
# print("Max Sequence Length:", max_sequence_length_x)
xtrain_np=xtrain.numpy()
xtest_np=xtest.numpy()


Input IDs size: torch.Size([30580, 2216])
Attention Mask size: torch.Size([30580, 2216])
Vocabulary Size: 33991


In [9]:
model=Sequential()
input_length=None
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=128,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=64)



Epoch 1/5


383/383 [==============================] - 1027s 3s/step - loss: 0.6935 - accuracy: 0.5012
Epoch 2/5
383/383 [==============================] - 1104s 3s/step - loss: 0.6933 - accuracy: 0.4947
Epoch 3/5
383/383 [==============================] - 1102s 3s/step - loss: 0.6932 - accuracy: 0.4989
Epoch 4/5
383/383 [==============================] - 1189s 3s/step - loss: 0.6932 - accuracy: 0.5026
Epoch 5/5
383/383 [==============================] - 851s 2s/step - loss: 0.6933 - accuracy: 0.4988


In [19]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

192/192 [==============================] - 46s 238ms/step - loss: 0.6931 - accuracy: 0.5046


[0.6931071877479553, 0.504578173160553]

In [20]:
model.save("dmodel1.keras")

In [21]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
ml = load_model("dmodel1.keras")
xnew = "We accomplish more by doing tasks that we are assigned with to complete We have always get something done if the tasks or assignments are done Take for example people that work jobs day and night they tend to get work or tasks or assignments done How do they get stuff done workers usually plan it out like steps to go though to complete them if they dont follow the steps like how are they going to complete the task or assignment to get it done Workers always need to plan it out before proceeding to the task or assignment to finish or complete For example Generic_Name wants to figure out how to complete his project he need to plan step by step and what materials he needs for the project He follows the step by step plan but realizes what he needs to do later on he gathers the materals and starts to complete his project just by always doing it All of us always have to something to complete any tasks or assignments to finish It s not going to work if we dont always doing something or plan it out it s the reason why if you wanna get it stuff done you have always do it in order complete it For example take school students for a example they tend to always to get work done so they can free time to themselves and usually making sure the answers they write down are correct to get good grades School students have determation to complete classwork they re assigned to so later on they don t have to complete it for homework at home for free time It s better to always complete work so you don t have to complete later on For example any job work usually has work to do like car manufactures they have a assembly to work on because it s where the cars are starting to be made by the workers in the assembly line They have to work the machines to put the parts on the cars that are being work on so it can shipped of to the car dealership Everything revolves around on always getting something on time so it doesn t become a problem later on Everyone like to job workers to school students alwayss have to something its why we tend to get stuff done Always accomplish it by doing it Reasons why we have to do it if we tend to get our task or assignments done then that work doesn t get it the way of what you are trying to on the next task Accomplishing that we are trying to get to is a good way to start off you see like CEOs of a company planning what they want done and to figure it how to accomplish it For example take CEOs they re good at getting stuff done and accomplished it pefectly cause they own businesses and tell his workers what food products or drinks that stores need for people to attract to buy their goods or clothes People always have to plan to work towards to they want to achieve or accomplish even for their future it s not going to work if you don t put the effort into the assignment or tasks to make it happpen You always have to make sure what you want and to make it happen so that your future has more opportuniites In conclusion Always put the effort into it so you can accomplish on any work task assignments that you can achieve later on in your life"
# xnew="Studying Science and History at Generic_School offers two very different experiences each with its own advantages Science is a constantly evolving field with new advances and techniques being discovered daily The school provides expert guidance and modern equipment making it an ideal place to explore the complex theories and technologies of the subject In contrast History offers an opportunity to delve into the past and investigate the origins of our civilization and culture Through lectures and field trips Generic_School s expert faculty provides students with a solid foundation of knowledge in the subject while also allowing them to study and discuss unique interpretations of traditional narratives Both Science and History have their advantages but in the end it comes down to each individual student s interests and goals If you are passionate about understanding the complexities of our world then studying Science is the perfect choice If on the other hand you enjoy delving into the past and uncovering the stories of our ancestors then History is the subject for you Whichever path you choose to take the experienced faculty and resources at Generic_School will provide the guidance you need to succeed and make the most of your studies"
# Tokenize the input text
xnew_tokens = tokenizer.encode(xnew,truncation=True,padding=True,return_tensors="np")

max_sequence_length = 2216
xnew_padded = pad_sequences(xnew_tokens, maxlen=max_sequence_length, padding='post', truncating='post')

# Use the padded tensor for prediction
prediction = ml.predict(xnew_padded)

# Set a threshold for classification
threshold = 0.5
prediction_value = prediction[0, 0]

# Check the prediction and print the result
if prediction_value >= threshold:
    print("1: It is an AI-generated text")
else:
    print("0: It is Human-generated text")


1/1 [==============================] - 0s 409ms/step
0: It is Human-generated text


In [13]:
import pandas as pd
data=pd.read_csv(r"B:\project\ai\ai_chunk_3.csv")
# essay=data['text'].tolist()
# tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)
data.isnull().sum()

text         0
generated    0
dtype: int64

In [14]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain_np=xtrain.numpy()
xtest_np=xtest.numpy()
xtrain_np1=xtrain_np
xtest_np1=xtest_np


In [15]:
import numpy as np
xtrain_np1 = np.array(xtrain_np1)

np.savetxt('B:/project/train/xtrain_np1.csv', xtrain_np1, delimiter=',')

In [16]:
xtest_np1=np.array(xtest_np1)
np.savetxt('B:/project/test/xtest_np1.csv', xtest_np1, delimiter=',')


In [17]:
ytest.to_csv("B:/project/ytest/ytest1.csv")

In [18]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
ytrain.to_csv("B:/project/ytrain/ytrain1.csv")
